In [1]:
# wangling
import numpy as np
import pandas as pd
import feather

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# geospatial
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

# others
import re
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

# load data

In [2]:
# police
path = "data/crime_counts/counts_binary_tract_id_1H.feather"
df = feather.read_dataframe(path)
df["datetime"] = pd.to_datetime(df["datetime"])
print(df.shape)
df.head()

(2203129, 3)


,tract_id,datetime,crime
0,101,2018-01-01 08:00:00,1
1,101,2018-01-01 09:00:00,0
2,101,2018-01-01 10:00:00,0
3,101,2018-01-01 11:00:00,0
4,101,2018-01-01 12:00:00,0


In [3]:
def add_geometry(df, crs):
    """
    return dataframe with geometry
    """
    # load corresponding census geo data
    if "tract_id" in df.columns:
        geodf_id = "name10"
        df_id = "tract_id"
        path = "data/census2010_ sf_tracks.geojson"
    elif "block_id" in df.columns:
        geodf_id = "blockce10"
        df_id = "block_id"
        path = "data/census2010_ sf_blocks.geojson"
    
    # load corresponding census geo data
    cen_geodf = gpd.read_file(path)
    cen_geodf = cen_geodf[['geometry', geodf_id]]
    
    # add geometory
    df = pd.merge(df, cen_geodf, left_on=df_id, right_on=geodf_id)
    df.drop(geodf_id, axis=1, inplace=True)
    geodf = gpd.GeoDataFrame(df, crs=crs)
    
    return geodf

In [4]:
# convert to geodataframe
crs = {'init':'epsg:4326'}
geodf = add_geometry(df, crs)
print(geodf.shape)
geodf.head()

(2203129, 4)


,tract_id,datetime,crime,geometry
0,101,2018-01-01 08:00:00,1,"(POLYGON ((-122.421076 37.812889, -122.420182 ..."
1,101,2018-01-01 09:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ..."
2,101,2018-01-01 10:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ..."
3,101,2018-01-01 11:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ..."
4,101,2018-01-01 12:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ..."


In [5]:
def exchange_coordinate(df, lon, lat, prefix):
    """
    return df with fixed coordinate
    """
    lon_fix_name = prefix + "_lon_fix"
    lat_fix_name = prefix + "_lat_fix"

    df[lon_fix_name] = df[lon]
    df[lat_fix_name] = df[lat]

    df.loc[df[lat] < 0, lon_fix_name] = df.loc[df[lat] < 0, lat_fix_name]
    df.loc[df[lat] < 0, lat_fix_name] = df.loc[df[lat] < 0, lon_fix_name]

    return df

In [6]:
def df_2_geodf(df, crs, lon, lat):
    """
    return geodataframe
    """
    geometry = [Point(xy) for xy in zip(df[lon], df[lat])]
    geodf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return geodf

In [7]:
def add_weather(geodf):
    """
    return dataframe with nearest weather station data
    """
    # load weather data
    weather = pd.read_csv('data/weather.csv')
    weather = exchange_coordinate(weather, lon="LONGITUDE", lat="LATITUDE", prefix="station")
    weather = df_2_geodf(weather, geodf.crs, lon="station_lon_fix", lat="station_lat_fix")
    
    # load sf data
    sf = gpd.read_file("data/census2010_ sf_tracks.geojson", crs=weather.crs)
    sf = sf[['geometry']]
    
    # extract only stations in SF
    weather_sf = gpd.sjoin(weather, sf, how="inner", op="intersects")
    weather_sf["date"] = pd.to_datetime(weather_sf["DATE"])
    weather_sf.set_index("date", inplace=True)

    # get average precipitation per day
    weather_day = weather_sf.resample("1D").agg({"PRCP": ["mean"],})
    weather_day.reset_index(inplace=True)
    weather_day.columns = ["date", "prcp"]
    
    # add precipitation
    geodf["date"] = pd.to_datetime(geodf["datetime"].dt.date)
    geodf = pd.merge(geodf, weather_day, how="left", on="date")
    
    return geodf

In [8]:
# add precipitation
geodf = add_weather(geodf)
geodf.head()

/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (19,21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,tract_id,datetime,crime,geometry,date,prcp
0,101,2018-01-01 08:00:00,1,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0
1,101,2018-01-01 09:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0
2,101,2018-01-01 10:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0
3,101,2018-01-01 11:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0
4,101,2018-01-01 12:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0


- The code below can get the nearest weahter station data.
- But it is computationally very intensive to calculate the nearest distance between polygon and point (point and point is much faster)
- So, the average of weather stations in SF is used in stead of the nearest value.

In [9]:
# def find_nearest_value(point, pts, geodf2, src_col):
#     """
#     return value of src_col in nearest point in geodf2 
#     """
#     nearest_point = nearest_points(point, pts)[1]
#     value = geodf2.loc[geodf2["geometry"] == nearest_point, src_col].values[0]
    
#     return value

# # add nearest weather station data
# pts = weather.geometry.unary_union
# geodf["prcp"] = geodf[:5].apply(lambda x: find_nearest_value(x.geometry, pts, weather, "PRCP"), axis=1)
# geodf["tavg"] = geodf[:5].apply(lambda x: find_nearest_value(x.geometry, pts, weather, "TAVG"), axis=1)

# feature engineering

## datetime

In [10]:
def convert_datetime(geodf):
    """
    return geodf with datetime columns
    """
    geodf["year"] = geodf.datetime.dt.year
    geodf["month"] = geodf.datetime.dt.month
    geodf["woy"] = geodf.datetime.dt.weekofyear 
    geodf["dow"] = geodf.datetime.dt.dayofweek
    geodf["weekend"] = geodf["dow"].apply(lambda x: 1 if x >=5 else 0) 
    geodf["hour"] = geodf.datetime.dt.hour
    
    return geodf

In [11]:
geodf = convert_datetime(geodf)
geodf.head()

,tract_id,datetime,crime,geometry,date,prcp,year,month,woy,dow,weekend,hour
0,101,2018-01-01 08:00:00,1,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,8
1,101,2018-01-01 09:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,9
2,101,2018-01-01 10:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,10
3,101,2018-01-01 11:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,11
4,101,2018-01-01 12:00:00,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,12


## exponentially weighted mean

- ewm weights more recent trend

In [12]:
def get_ewm(series, alpha, adjust=True, timesteps=1):
    """
    return series with exponential weighted mean
    """
    # shift data to avoid leakage
    ewm = series.shift(timesteps).ewm(alpha=alpha, adjust=adjust).mean()
    
    return ewm

In [13]:
def get_grouped_ewm(geodf, groupby, alpha, adjust=True, timesteps=1):
    """
    return dataframe with exponentaial weighted mean
    """
    # calculate exponential weighted mean by each groupby unit
    roll = geodf.groupby(groupby).apply(lambda x: get_ewm(x.crime, alpha, adjust, timesteps))
    geodf["ewm_" + str(alpha)] = roll.sort_index(level = ["tract_id", "datetime"]).values
    
    return geodf

In [22]:
# add exponential weighted mean
geodf = get_grouped_ewm(geodf.set_index("datetime"),
                        groupby=["tract_id"], alpha=0.5,
                        adjust=True, timesteps=1)
geodf.reset_index(inplace=True)
geodf.head()

,datetime,tract_id,crime,geometry,date,prcp,year,month,woy,dow,weekend,hour,ewm_0.5,ewm_0.8
0,2018-01-01 08:00:00,101,1,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,8,NaN,NaN
1,2018-01-01 09:00:00,101,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,9,1.000000,1.000000
2,2018-01-01 10:00:00,101,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,10,0.333333,0.166667
3,2018-01-01 11:00:00,101,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,11,0.142857,0.032258
4,2018-01-01 12:00:00,101,0,"(POLYGON ((-122.421076 37.812889, -122.420182 ...",2018-01-01,0.0,2018,1,1,0,0,12,0.066667,0.006410


# save

In [25]:
# geometry cannot be included in feather format
geodf.drop("geometry", axis=1, inplace=True)

path = "features/features_binary_tract_id_1H.feather"
geodf.to_feather(path)